In [86]:
import pandas as pd
import numpy as np
import os

In [87]:
# creating the dataframes
schoolsdf = pd.read_csv(os.path.join("raw_data","schools_complete.csv"))
studentsdf =  pd.read_csv(os.path.join("raw_data","students_complete.csv"))
schoolsdf.describe()


,School ID,size,budget
count,15.000000,15.000000,1.500000e+01
mean,7.000000,2611.333333,1.643295e+06
std,4.472136,1420.915282,9.347763e+05
min,0.000000,427.000000,2.480870e+05
25%,3.500000,1698.000000,1.046265e+06
50%,7.000000,2283.000000,1.319574e+06
75%,10.500000,3474.000000,2.228999e+06
max,14.000000,4976.000000,3.124928e+06


In [88]:
# This cell is for learning about the Schools Data
#schoolsdf.describe()

schoolsdf = schoolsdf.rename(columns = {'name':'school'})
schoolsdf

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [89]:
#This cell is for learningabout the Students Data
#studentsdf.describe()
studentsdf.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score'],
      dtype='object')

In [90]:
studentsdf.describe()


,Student ID,reading_score,math_score
count,39170.000000,39170.00000,39170.000000
mean,19584.500000,81.87784,78.985371
std,11307.549359,10.23958,12.309968
min,0.000000,63.00000,55.000000
25%,9792.250000,73.00000,69.000000
50%,19584.500000,82.00000,79.000000
75%,29376.750000,91.00000,89.000000
max,39169.000000,99.00000,99.000000


In [91]:
# District Summary
TotalSchools = schoolsdf['School ID'].count()
TotalStudents = studentsdf['Student ID'].count()
Totalbudget = schoolsdf['budget'].sum()
AvgMathScore = studentsdf['math_score'].mean()
AvgReadingScore = studentsdf['reading_score'].mean()
PerMathPass= (len(studentsdf.loc[studentsdf['math_score'] >65])/TotalStudents)*100
PerReadingPass = (len(studentsdf.loc[studentsdf['reading_score'] >65])/TotalStudents)*100
OverallPassingRate =(PerMathPass+PerReadingPass)/2


DistrictSummary= {'Total Schools': TotalSchools,'Total Students' : TotalStudents,'Total budget': Totalbudget,
                  'Average Math Score': AvgMathScore, 'Average Reading Score':AvgReadingScore,
                  '% Passing Math': PerMathPass, '% Passing Reading' :PerReadingPass, 'Overall Passing Rate' : OverallPassingRate}

DistrictSummaryDf = pd.DataFrame(DistrictSummary,columns = DistrictSummary.keys(),index=[0])
DistrictSummaryDf

,Total Schools,Total Students,Total budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,83.112076,94.263467,88.687771


In [92]:
#School Summary
#Final columns in the report
SummaryCol = ['School Name','School Type','Total Students','Total School Budget',
              'Per Student Budget','Average Math Score','Average Reading Score',
              '% Passing Math','% Passing Reading', 'Overall Passing Rate']


# We are creating this report in 4 steps.
#Step 1 : Group by the students dataframe by school and aggregate as many fields as you ca
schoolSumDf = studentsdf.groupby("school").agg({'reading_score':['mean'],
                                                'math_score':['mean']})
schoolSumDf.reset_index(inplace = True)
schoolSumDf.columns=[''.join(col) for col in schoolSumDf.columns ]
schoolSumDf = schoolSumDf.rename(columns = {'reading_scoremean': 'Average Reading Score','math_scoremean': 'Average Math Score'})


#Step 2 : Create 2 dataframes with number of students passing math and reading
StudentMathPassDf = studentsdf.loc[studentsdf['math_score'] >65,['school','math_score']]
StudentMathPassDf=StudentMathPassDf.groupby("school").count()
#StudentMathPassDf.reset_index(inplace = True)
StudentMathPassDf.reset_index(inplace =True)
pd.DataFrame(StudentMathPassDf.to_records()) 
StudentMathPassDf = StudentMathPassDf.rename(columns = {'math_score': 'NumMathPassed'})

StudentReadingPassDf = studentsdf.loc[studentsdf['reading_score'] >65,['school','reading_score']]
StudentReadingPassDf=StudentReadingPassDf.groupby("school").count()
StudentReadingPassDf.reset_index(inplace =True)
pd.DataFrame(StudentReadingPassDf.to_records())
StudentReadingPassDf = StudentReadingPassDf.rename(columns = {'reading_score': "NumReadingPassed"})
                                        
#Step 3 combine the schoolsDf,SchoolSumDf,StudentMathPassDf,StudentReadingPassDf 
merdf1 = pd.merge(schoolsdf,schoolSumDf,how = 'outer', on='school')
merdf1.reset_index()

merdf2 = pd.merge(StudentMathPassDf,StudentReadingPassDf,on='school')
merdf2.reset_index()

merdf3 = pd.merge(merdf1,merdf2,on='school')
merdf3.reset_index()
FinalDf = merdf3
FinalDf['Per Student Budget'] = FinalDf['budget']/FinalDf['size']
FinalDf['% Passing Math'] = (FinalDf['NumMathPassed']/FinalDf['size'])*100
FinalDf['% Passing Reading'] = (FinalDf['NumReadingPassed']/FinalDf['size'])*100
FinalDf['Overall Passing Rate'] = (((FinalDf['NumReadingPassed']/FinalDf['size'])*100)+
                                ((FinalDf['NumMathPassed']/FinalDf['size'])*100))/2
FinalDf['Per Student Budget'] = FinalDf['budget']/FinalDf['size']

ReportDf = FinalDf.rename(columns = {'school': 'School Name','type': 'School Type','size' :'Total Students','budget':'Total School Budget'})
schoolSumDf = ReportDf[SummaryCol]
schoolSumDf


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,75.282825,91.772369,83.527597
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,74.872838,91.895558,83.384198
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,100.000000,100.000000,100.000000
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,75.469256,91.456311,83.462783
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,100.000000,100.000000,100.000000
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,100.000000,100.000000,100.000000
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,100.000000,100.000000,100.000000
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,75.602894,91.901125,83.752010
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,100.000000,100.000000,100.000000
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,100.000000,100.000000,100.000000


In [93]:
#Top Performing Schools
TopPerformingSchoolsDf = schoolSumDf.nlargest(5,columns = ['Overall Passing Rate'])
TopPerformingSchoolsDf

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,100.0,100.0,100.0
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,100.0,100.0,100.0
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,100.0,100.0,100.0
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,100.0,100.0,100.0
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,100.0,100.0,100.0


In [94]:
#Bottom 5 Performing Schools
LeastPerformingSchoolsDf = schoolSumDf.nsmallest(5,columns = ['Overall Passing Rate'])
LeastPerformingSchoolsDf

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,74.872838,91.895558,83.384198
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,75.469256,91.456311,83.462783
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,76.195692,90.799562,83.497627
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,75.282825,91.772369,83.527597
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,75.543886,91.522881,83.533383


In [95]:
# Reading and Math Score by Grade
studentsdf.head()
# Select students by grade, group them by school and get the average reading scores

students9thDF = (studentsdf.loc[studentsdf['grade'] == "9th"]).groupby(["school"])[["reading_score","math_score"]].mean()
students9thDF.reset_index(inplace =True)
pd.DataFrame(students9thDF.to_records())
students9thDF = students9thDF.rename(columns = {'reading_score': "9th_Reading",'math_score' : "9th_Math"})


students10thDF = (studentsdf.loc[studentsdf['grade'] == "10th"]).groupby(["school"])[["reading_score","math_score"]].mean()
students10thDF.reset_index(inplace =True)
pd.DataFrame(students10thDF.to_records())
students10thDF = students10thDF.rename(columns = {'reading_score': "10th_Reading",'math_score' : "10th_Math"})


students11thDF = (studentsdf.loc[studentsdf['grade'] == "11th"]).groupby(["school"])[["reading_score","math_score"]].mean()
students11thDF.reset_index(inplace =True)
pd.DataFrame(students11thDF.to_records())
students11thDF = students11thDF.rename(columns = {'reading_score': "11th_Reading",'math_score' : "11th_Math"})


students12thDF = (studentsdf.loc[studentsdf['grade'] == "12th"]).groupby(["school"])[["reading_score","math_score"]].mean()
students12thDF.reset_index(inplace =True)
pd.DataFrame(students12thDF.to_records())
students12thDF = students12thDF.rename(columns = {'reading_score': "12th_Reading",'math_score' : "12th_Math"})

#merge 9,10
mer910df = pd.merge(students9thDF,students10thDF,how = 'outer', on='school')
mer910df.reset_index()

# merge 11,12
mer1112df = pd.merge(students11thDF,students12thDF,on='school')
mer1112df.reset_index()

#Merge above two
studentAvgScoresByGrade = pd.merge(mer910df,mer1112df,on='school')
studentAvgScoresByGrade.reset_index()

studentAvgScoresByGrade



,school,9th_Reading,9th_Math,10th_Reading,10th_Math,11th_Reading,11th_Math,12th_Reading,12th_Math
0,Bailey High School,81.303155,77.083676,80.907183,76.996772,80.945643,77.515588,80.912451,76.492218
1,Cabrera High School,83.676136,83.094697,84.253219,83.154506,83.788382,82.765560,84.287958,83.277487
2,Figueroa High School,81.198598,76.403037,81.408912,76.539974,80.640339,76.884344,81.384863,77.151369
3,Ford High School,80.632653,77.361345,81.262712,77.672316,80.403642,76.918058,80.662338,76.179963
4,Griffin High School,83.369193,82.044010,83.706897,84.229064,84.288089,83.842105,84.013699,83.356164
5,Hernandez High School,80.866860,77.438495,80.660147,77.337408,81.396140,77.136029,80.857143,77.186567
6,Holden High School,83.677165,83.787402,83.324561,83.429825,83.815534,85.000000,84.698795,82.855422
7,Huang High School,81.290284,77.027251,81.512386,75.908735,81.417476,76.446602,80.305983,77.225641
8,Johnson High School,81.260714,77.187857,80.773431,76.691117,80.616027,77.491653,81.227564,76.863248
9,Pena High School,83.807273,83.625455,83.612000,83.372000,84.335938,84.328125,84.591160,84.121547


In [99]:
#Reading Scores by Grade 
AvgReadingScoresByGrade=studentAvgScoresByGrade.loc[:,['school','9th_Reading','10th_Reading','11th_Reading','12th_Reading']]
AvgReadingScoresByGrade = AvgReadingScoresByGrade.rename(columns = {'school': 'School Name','9th_Reading': '9th','10th_Reading': '10th','11th_Reading': '11th','12th_Reading': '12th'})
AvgReadingScoresByGrade                                                 

,School Name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [100]:
#Reading Scores by Grade 
AvgMathScoresByGrade=studentAvgScoresByGrade.loc[:,['school','9th_Math','10th_Math','11th_Math','12th_Math']]
AvgMathScoresByGrade = AvgMathScoresByGrade.rename(columns = {'school': 'School Name','9th_Math': '9th','10th_Math': '10th','11th_Math': '11th','12th_Math': '12th'})
AvgMathScoresByGrade 

,School Name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [148]:
# Scores by School Spending
# In order to create the bins based on avegrage spending ranges we need 
#to find the maximum per student budget 

MaxPerStudentBudget = schoolSumDf['Per Student Budget'].max()
MinPerStudentBudget = schoolSumDf['Per Student Budget'].min()

binInterval = ((MaxPerStudentBudget-MinPerStudentBudget)/4)


# Now divide that into 4 bins
SchoolSpendingBins = [0,MinPerStudentBudget+ binInterval,MinPerStudentBudget+binInterval*2,MinPerStudentBudget+binInterval*3, MaxPerStudentBudget]

#TestDf12 = schoolSumDf
#TestDf['bin'] = pd.cut(schoolSumDf['Per Student Budget'],SchoolSpendingBins)
#TestDF12
# convert the % and averages in schoolSumDf into whole numbers
schoolSumDfNum = schoolSumDf
schoolSumDfNum ['TotalMathScore'] = schoolSumDfNum['Average Math Score'] *schoolSumDfNum['Total Students']
schoolSumDfNum ['TotalReadingScore'] = schoolSumDfNum['Average Reading Score'] *schoolSumDfNum['Total Students']
schoolSumDfNum ['NumReadingPassed'] = (schoolSumDfNum['Average Reading Score'] *schoolSumDfNum['Total Students'])/100
schoolSumDfNum ['NumMathPassed'] = (schoolSumDfNum['Average Math Score'] *schoolSumDfNum['Total Students'])/100

schoolSumDfNum['Spending Per Student'] = pd.cut(schoolSumDfNum['Per Student Budget'], SchoolSpendingBins)
SummaryBySpendingDf1 = schoolSumDfNum.groupby(['Spending Per Student'])[['Total Students','TotalMathScore','TotalReadingScore','NumReadingPassed','NumMathPassed']].sum()

SummaryBySpendingDf1 ['Average Math Score']=SummaryBySpendingDf1 ['TotalMathScore']/SummaryBySpendingDf1 ['Total Students']
SummaryBySpendingDf1 ['Average Reading Score']=SummaryBySpendingDf1 ['TotalReadingScore']/SummaryBySpendingDf1 ['Total Students']
SummaryBySpendingDf1 ['% Passing Math']= (SummaryBySpendingDf1 ['NumMathPassed']/SummaryBySpendingDf1 ['Total Students'])*100
SummaryBySpendingDf1 ['% Passing Reading']= (SummaryBySpendingDf1 ['NumReadingPassed']/SummaryBySpendingDf1 ['Total Students'])*100

SummaryBySpendingDf = SummaryBySpendingDf1.loc[:,['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading']]
SummaryBySpendingDf ['Over all Passing Rate'] = (SummaryBySpendingDf['% Passing Math']+SummaryBySpendingDf['% Passing Math'])/2
SummaryBySpendingDf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Over all Passing Rate
Spending Per Student,,,,,
"(0.0, 597.25]",83.363065,83.964039,83.363065,83.964039,83.363065
"(597.25, 616.5]",83.529196,83.838414,83.529196,83.838414,83.529196
"(616.5, 635.75]",78.484327,81.667908,78.484327,81.667908,78.484327
"(635.75, 655.0]",77.418997,81.147112,77.418997,81.147112,77.418997


In [119]:
schoolsdf


,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858
